## 2. Data preprocessing

### Import neccessary libraries

In [26]:
import pandas as pd
import numpy as np
import re
import json
from typing import List, Dict

Read collected datasets and view their shapes

In [27]:
steamspy_df = pd.read_csv('data/steamspy_data.csv')
steamstore_df = pd.read_csv('data/steamstore_data.csv')
steamspy_df.head()


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,NaN,1962552,441773,0,"200,000,000 .. 500,000,000",0,0,0,0,0.0,0.0,0.0,603767
1,730,Counter-Strike: Global Offensive,Valve,Valve,NaN,7350515,1107856,0,"100,000,000 .. 200,000,000",0,0,0,0,0.0,0.0,0.0,1068631
2,578080,PUBG: BATTLEGROUNDS,PUBG Corporation,"KRAFTON, Inc.",NaN,1464508,1016035,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,461831
3,1623730,Palworld,Pocketpair,Pocketpair,NaN,336427,20836,0,"50,000,000 .. 100,000,000",0,0,0,0,2249.0,2999.0,25.0,171069
4,1172470,Apex Legends,Respawn,Electronic Arts,NaN,656605,315391,0,"50,000,000 .. 100,000,000",0,0,0,0,0.0,0.0,0.0,79726


In [28]:
steamstore_df.head()

,appid,name,languages,metacritic,genres,release_date,required_age,dlc
0,570,Dota 2,"Bulgarian, Czech, Danish, Dutch, English<stron...","{'score': 90, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '9 Jul, 2013'}",0,"[1241930, 652720]"
1,730,Counter-Strike 2,"Czech, Danish, Dutch, English<strong>*</strong...",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '21 Aug, 2012'}",0,[2678630]
2,578080,PUBG: BATTLEGROUNDS,"English, Korean, Simplified Chinese, French, G...",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '21 Dec, 2017'}",0,NaN
3,1623730,Palworld,"English, Simplified Chinese, Traditional Chine...",NaN,"[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '18 Jan, 2024'}",0,[2771110]
4,1172470,Apex Legends™,"English<strong>*</strong>, French<strong>*</st...","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}, {'id': ...","{'coming_soon': False, 'date': '4 Nov, 2020'}",0,NaN


Merge the two data together using appid and name

In [29]:
merged_df = pd.merge(steamspy_df, steamstore_df, on=['appid', 'name'], how='inner')
merged_df.drop(['developer','publisher','score_rank','metacritic'], axis=1, inplace=True)
print(f"Number of games in merged_df: {merged_df.shape[0]}")
print(merged_df.isnull().sum())


Number of games in merged_df: 18605
appid                  0
name                   0
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                  0
initialprice           0
discount               0
ccu                    0
languages             10
genres                47
release_date          24
required_age           0
dlc                12684
dtype: int64


Preprocessing data

In [ ]:

# Feature Engineering

# 1. owners_lower_bound and owners_log
def owners_to_numeric(owners):
    # If already numeric, return as is
    if isinstance(owners, (int, float)):
        return owners
        
    # Handle string case
    if isinstance(owners, str):
        lower_bound = owners.split(" .. ")[0]
        return int(lower_bound.replace(",", ""))
        
    # Handle unexpected cases
    return 0


merged_df['owners'] = merged_df['owners'].apply(owners_to_numeric)
merged_df['owners_log'] = np.log1p(merged_df['owners'])

# 2. dlc_count and dlc_count_log
merged_df['dlc_count'] = merged_df['dlc'].apply(lambda x: 0 if pd.isnull(x) or x == '' else len(x.split(',')))
merged_df['dlc_count_log'] = np.log1p(merged_df['dlc_count'])

# 3. release age
def extract_year(release_date_str):
    if pd.isnull(release_date_str):
        return None
    match = re.search(r"(19|20)\d{2}", release_date_str)
    if match:
        return int(match.group(0))
    else:
        return None

merged_df['release_year'] = merged_df['release_date'].apply(extract_year)
merged_df['release_age'] = 2024 - merged_df['release_year']

# 4. required_age
merged_df['required_age'] = pd.to_numeric(merged_df['required_age'], errors='coerce')
merged_df['required_age'] = merged_df['required_age'].fillna(0).astype(int)

# 5. languages_count
def count_languages(languages_str):
    if pd.isnull(languages_str) or not isinstance(languages_str, str):
        return 0
    languages_str = re.sub('<[^<]+?>', '', languages_str)  # Remove HTML tags
    return len(languages_str.split(','))

merged_df['languages_count'] = merged_df['languages'].apply(count_languages)

# 6. Genres
def extract_genres(genres_str):
    if pd.isnull(genres_str):
        return []
    try:
        return genres_str.lower().split(',')
    except:
        return []
merged_df['genre_list'] = merged_df['genres'].apply(extract_genres)

# List of top genres to create boolean columns for
top_genres = ['indie', 'action', 'casual', 'adventure', 'simulation',
              'rpg', 'strategy', 'sports', 'racing','massively multiplayer']

# Create boolean columns for each top genre
for genre in top_genres:
    merged_df[f'genre_{genre}'] = merged_df['genre_list'].apply(lambda x: int(genre in x))
    
# 7. Userscore
merged_df['userscore'] = merged_df['positive'] / (merged_df['positive'] + merged_df['negative'])
merged_df = merged_df.drop(['positive', 'negative'], axis=1)

# 8. Concurrent users
median_of_ccu = int(merged_df[merged_df['ccu'] > 0]['ccu'].median())
merged_df.loc[merged_df['ccu'] == 0, 'ccu'] = median_of_ccu
merged_df['ccu_log'] = np.log1p(merged_df['ccu'])

# 9. Median playtime (API error)

# # Rename columns for clarity
# processed_df = processed_df.rename(columns={'median_forever': 'median_playtime'})

# # Calculate median and replace zeros for playtime
# median_of_median_playtime = int(processed_df[processed_df['median_playtime'] > 0]['median_playtime'].median())
# processed_df.loc[processed_df['median_playtime'] == 0, 'median_playtime'] = median_of_median_playtime

# # Apply log transformation to playtime
# processed_df['median_playtime_log'] = np.log1p(processed_df['median_playtime'])
# processed_df.drop(columns=['median_playtime'], inplace=True)


# Select relevant columns
selected_columns = ['appid', 'name', 'userscore', 'owners', 'owners_log', 'release_year', 'release_age', 
                    'required_age', 'dlc_count', 'dlc_count_log', 'languages_count', 'price', 'ccu', 'ccu_log'
                    ]
selected_columns += [f'genre_{genre}' for genre in top_genres]
processed_df = merged_df[selected_columns].copy()
processed_df = processed_df.rename(columns={'genre_massively multiplayer': 'genre_massively_multiplayer'})

# Remove duplicates based on appid (keeping the first occurrence)
processed_df = processed_df.drop_duplicates(subset=['appid'], keep='first')

processed_df = processed_df.dropna()
# Save processed data to a CSV file
processed_df.to_csv('data/processed_data.csv', index=False)

In [31]:
processed_df.head()


,appid,name,userscore,owners,owners_log,release_year,release_age,required_age,dlc_count,dlc_count_log,...,genre_indie,genre_action,genre_casual,genre_adventure,genre_simulation,genre_rpg,genre_strategy,genre_sports,genre_racing,genre_massively_multiplayer
0,570,Dota 2,0.816259,200000000,19.113828,2013.0,11.0,0,2,1.098612,...,0,0,0,0,0,0,0,0,0,0
1,578080,PUBG: BATTLEGROUNDS,0.590398,50000000,17.727534,2017.0,7.0,0,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,1623730,Palworld,0.941679,50000000,17.727534,2024.0,0.0,0,1,0.693147,...,0,0,0,0,0,0,0,0,0,0
3,1063730,New World: Aeternum,0.686755,50000000,17.727534,2021.0,3.0,0,1,0.693147,...,0,0,0,0,0,0,0,0,0,0
4,2358720,Black Myth: Wukong,0.965987,50000000,17.727534,2024.0,0.0,0,2,1.098612,...,0,0,0,0,0,0,0,0,0,0
